In [1]:
import os

import pyspark.pandas as pspd
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, collect_list, udf
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType

In [2]:
spark = SparkSession.builder.appName(
    'Collaborative Filtering').config(
    "spark.executor.memory", "6g").master(
    'local[*]').config(
    'spark.driver.extraClassPath', 'C:/Users/Masum Billal/Downloads/Programs/mysql-connector-java-8.0.30.jar').getOrCreate()

In [3]:
host = os.environ['MYSQL_HOST']
properties = {
    'user': os.environ['MYSQL_USER'],
    'password': os.environ['MYSQL_PASSWORD'],
    'driver': 'com.mysql.jdbc.Driver',
    'dbtable': 'manager_team',
}
url = f'jdbc:mysql://{host}/football_manager'

rdf = spark.read.jdbc(url=url, properties=properties, table='manager_team')
rdf.show()

+---+--------------------+--------------------+------+-----------+-------+--------+-------+-----------+-----------------------+--------+---------+----------+--------+
| id|          created_at|          updated_at|  name|num_players| budget|   value|earning|has_manager|starting_manager_salary|existing|league_id|manager_id|owner_id|
+---+--------------------+--------------------+------+-----------+-------+--------+-------+-----------+-----------------------+--------+---------+----------+--------+
|  1|2022-07-20 06:48:...|2022-07-20 06:48:...|Team 1|         20|5000000|20000000|      0|       true|                  10000|    true|        1|         1|       1|
+---+--------------------+--------------------+------+-----------+-------+--------+-------+-----------+-----------------------+--------+---------+----------+--------+



In [4]:
sc = spark.sparkContext

In [5]:
schema = StructType([
    StructField('userId', IntegerType(), nullable=True),
    StructField('movieId', IntegerType(), nullable=True),
    StructField('rating', FloatType(), nullable=True),
    StructField('timestamp', IntegerType(), nullable=True),
])

In [6]:
import pandas as pd
df = pd.DataFrame([{'a': 10, 'b': 1}, {'a': 10, 'b': 0}])
df

,a,b
0,10,1
1,10,0


In [7]:
df = df.groupby(by='a', as_index=False)[['b']].agg(lambda x: list(x))
df

,a,b
0,10,"[1, 0]"


In [8]:
# rdd = sc.textFile('ratings.csv')
# rdd.collect()

In [9]:
# headers = rdd.first()
# rdd = rdd.filter(lambda x: x != headers)
# rdd.collect()

In [10]:
%%time
rdf = spark.read.options(header= True,).schema(schema=schema).csv('ratings.csv')
rdf

CPU times: total: 0 ns
Wall time: 54.5 ms


DataFrame[userId: int, movieId: int, rating: float, timestamp: int]

In [11]:
rdf.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [12]:
rdf.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: integer (nullable = true)



In [13]:
rdf.rdd.getNumPartitions()

1

In [14]:
rdf.count()

100836

In [15]:
rdf.select('userId', 'movieId', 'rating').show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



In [16]:
rdf.select(rdf.userId, rdf.movieId, rdf.rating).show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



In [17]:
def normalize(rdf, column):
    mean = rdf.agg(avg(col(column)))
    mean = mean.collect()[0][0]
    rdf = rdf.withColumn(column+'_new', col(column)-mean)
    
    return rdf

In [18]:
df = rdf.pandas_api()
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [19]:
rdf = normalize(rdf, 'rating')
rdf.show()

+------+-------+------+---------+------------------+
|userId|movieId|rating|timestamp|        rating_new|
+------+-------+------+---------+------------------+
|     1|      1|   4.0|964982703| 0.498443016383038|
|     1|      3|   4.0|964981247| 0.498443016383038|
|     1|      6|   4.0|964982224| 0.498443016383038|
|     1|     47|   5.0|964983815| 1.498443016383038|
|     1|     50|   5.0|964982931| 1.498443016383038|
|     1|     70|   3.0|964982400|-0.501556983616962|
|     1|    101|   5.0|964980868| 1.498443016383038|
|     1|    110|   4.0|964982176| 0.498443016383038|
|     1|    151|   5.0|964984041| 1.498443016383038|
|     1|    157|   5.0|964984100| 1.498443016383038|
|     1|    163|   5.0|964983650| 1.498443016383038|
|     1|    216|   5.0|964981208| 1.498443016383038|
|     1|    223|   3.0|964980985|-0.501556983616962|
|     1|    231|   5.0|964981179| 1.498443016383038|
|     1|    235|   4.0|964980908| 0.498443016383038|
|     1|    260|   5.0|964981680| 1.4984430163

In [20]:
df['rating_new'] = df['rating']-df['rating'].mean()
df.head()

,userId,movieId,rating,timestamp,rating_new
0,1,1,4.0,964982703,0.498443
1,1,3,4.0,964981247,0.498443
2,1,6,4.0,964982224,0.498443
3,1,47,5.0,964983815,1.498443
4,1,50,5.0,964982931,1.498443


In [21]:
sdf = df.to_spark()
sdf.show()

+------+-------+------+---------+------------------+
|userId|movieId|rating|timestamp|        rating_new|
+------+-------+------+---------+------------------+
|     1|      1|   4.0|964982703| 0.498443016383038|
|     1|      3|   4.0|964981247| 0.498443016383038|
|     1|      6|   4.0|964982224| 0.498443016383038|
|     1|     47|   5.0|964983815| 1.498443016383038|
|     1|     50|   5.0|964982931| 1.498443016383038|
|     1|     70|   3.0|964982400|-0.501556983616962|
|     1|    101|   5.0|964980868| 1.498443016383038|
|     1|    110|   4.0|964982176| 0.498443016383038|
|     1|    151|   5.0|964984041| 1.498443016383038|
|     1|    157|   5.0|964984100| 1.498443016383038|
|     1|    163|   5.0|964983650| 1.498443016383038|
|     1|    216|   5.0|964981208| 1.498443016383038|
|     1|    223|   3.0|964980985|-0.501556983616962|
|     1|    231|   5.0|964981179| 1.498443016383038|
|     1|    235|   4.0|964980908| 0.498443016383038|
|     1|    260|   5.0|964981680| 1.4984430163

C:\Program Files\Python39\lib\site-packages\pyspark\pandas\utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [22]:
rdf.filter((rdf.rating >= 4) & (rdf.rating_new > 0)).show()

+------+-------+------+---------+-----------------+
|userId|movieId|rating|timestamp|       rating_new|
+------+-------+------+---------+-----------------+
|     1|      1|   4.0|964982703|0.498443016383038|
|     1|      3|   4.0|964981247|0.498443016383038|
|     1|      6|   4.0|964982224|0.498443016383038|
|     1|     47|   5.0|964983815|1.498443016383038|
|     1|     50|   5.0|964982931|1.498443016383038|
|     1|    101|   5.0|964980868|1.498443016383038|
|     1|    110|   4.0|964982176|0.498443016383038|
|     1|    151|   5.0|964984041|1.498443016383038|
|     1|    157|   5.0|964984100|1.498443016383038|
|     1|    163|   5.0|964983650|1.498443016383038|
|     1|    216|   5.0|964981208|1.498443016383038|
|     1|    231|   5.0|964981179|1.498443016383038|
|     1|    235|   4.0|964980908|0.498443016383038|
|     1|    260|   5.0|964981680|1.498443016383038|
|     1|    333|   5.0|964981179|1.498443016383038|
|     1|    349|   4.0|964982563|0.498443016383038|
|     1|    

In [23]:
df[df['rating'] >= 4].head()

,userId,movieId,rating,timestamp,rating_new
0,1,1,4.0,964982703,0.498443
1,1,3,4.0,964981247,0.498443
2,1,6,4.0,964982224,0.498443
3,1,47,5.0,964983815,1.498443
4,1,50,5.0,964982931,1.498443


In [24]:
rdf.select('rating').distinct().show()

+------+
|rating|
+------+
|   5.0|
|   2.5|
|   2.0|
|   3.0|
|   1.5|
|   0.5|
|   3.5|
|   1.0|
|   4.5|
|   4.0|
+------+



In [25]:
rdf.select('userId').distinct().count()

610

In [26]:
rdf.dropDuplicates(['userId', 'movieId']).count()

100836

In [27]:
rdf.count()

100836

In [28]:
group = rdf.groupBy('userId').agg(collect_list('movieId'), collect_list('rating'))
group = group.withColumnRenamed('collect_list(movieId)', 'movieId')
group = group.withColumnRenamed('collect_list(rating)', 'rating')

def get_avg(x):
    if len(x) < 1:
        return 0
    return sum(x)/len(x)


_udf = udf(lambda x: get_avg(x), FloatType())
group = group.withColumn('avg_rating', _udf(group.rating))
group.show()

Py4JJavaError: An error occurred while calling o429.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 39.0 failed 1 times, most recent failure: Lost task 0.0 in stage 39.0 (TID 27) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:855)
	at org.apache.spark.rdd.RDD.$anonfun

In [ ]:
group = group.pandas_api()
group.head()

In [ ]:
rdd_new = group.to_spark()
rdd_new.show()

In [ ]:
rdf_cached = rdf.alias('rdf_cached')
rdf_cached.cache()

In [ ]:
rdf_cached.show()